In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
##### for dirname, _, filenames in os.walk('/kaggle/input'):
##### 上一行用于提交Kaggle时使用
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./kaggle/input\us-patent-phrase-to-phrase-matching\sample_submission.csv
./kaggle/input\us-patent-phrase-to-phrase-matching\test.csv
./kaggle/input\us-patent-phrase-to-phrase-matching\train.csv


# 正文

In [2]:
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        if filename == 'train.csv':
            train_data = pd.read_csv(os.path.join(dirname, filename))
        elif filename == 'test.csv':
            test_data = pd.read_csv(os.path.join(dirname, filename))
        elif filename == 'sample_submission.csv':
            sample_submission = pd.read_csv(os.path.join(dirname, filename))

train_data

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [3]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from torch import nn
from torch.utils.data import DataLoader

import math
import random
from itertools import chain
from tqdm import tqdm

## 训练1

### 数据预处理

In [4]:
processed_data = []
for row in train_data.index:
    inp_example = InputExample(texts=[train_data.loc[row, 'anchor'],
                                     train_data.loc[row, 'target']],
                              label=float(train_data.loc[row, 'score']))
    processed_data.append(inp_example)
    
# From https://blog.csdn.net/weixin_44843824/article/details/108257594
def subset(alist, idxs):
    '''
    select the elements with index idx from alist
    '''
    sub_list = []
    for idx in idxs:
        sub_list.append(alist[idx])

    return sub_list

def split_list(alist, group_num=4, shuffle=True, retain_left=False):
    '''
    Divide alist into group_num number of groups.
    shuffle: whether to randomize the order
    retain_left: whether the residual elements become a new group
    '''

    index = list(range(len(alist)))

    if shuffle: 
        random.shuffle(index) 
    
    elem_num = len(alist) // group_num
    sub_lists = {}
    
    for idx in range(group_num):
        start, end = idx*elem_num, (idx+1)*elem_num
        sub_lists['set' + str(idx)] = subset(alist, index[start:end])
    
    if retain_left and group_num * elem_num != len(index):
        sub_lists['set' + str(idx + 1)] = subset(alist, index[end:])
    
    # Return dict and list of values for the selection of the required result
    return sub_lists, list(sub_lists.values())

_, divided_data = split_list(processed_data, 
                             group_num=5, 
                             shuffle=True,
                             retain_left=False)

train_processed_data = [x for l in divided_data[:-1] for x in l]
valid_processed_data = divided_data[-1]

### 模型参数

In [5]:
model_name = "all-mpnet-base-v2"
batch_size = 16
epoch_num = 4

In [ ]:
def train_model(training_data, validation_data, model_name, batch_size, epoch_num):
    print("Loading model...")
    model = SentenceTransformer(model_name)

    train_dataloader = DataLoader(training_data,
                                  shuffle=True,
                                  batch_size=batch_size)
    train_loss = losses.CosineSimilarityLoss(model)
    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(validation_data)
    warmup_steps = math.ceil(len(train_dataloader) * epoch_num * 0.1) # 10% of train data for warm-up
    
    print("Fit model...")
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              epochs=epoch_num,
              evaluator=evaluator,
              warmup_steps=warmup_steps)
    return model

model = train_model(train_processed_data, valid_processed_data, model_name, batch_size, epoch_num)

Loading model...
Fit model...


C:\Users\Administrator\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1824 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1824 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1824 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1824 [00:00<?, ?it/s]

In [ ]:
def test_score(model, test_phrases):
    embeddings1 = model.encode(test_phrases.loc[:, 'anchor'], convert_to_tensor=True)
    embeddings2 = model.encode(test_phrases.loc[:, 'target'], convert_to_tensor=True)
    output_scores = [util.cos_sim(embeddings1[i], embeddings2[i]).item() for i in range(len(test_phrases))]
    
    return output_scores

output_scores = test_score(model, train_data)

In [ ]:
def to_closest(array):
    interval = [-float('inf'), 0.125, 0.375, 0.625, 0.875, float('inf')]
    new_array = []
    for x in array:
        for i in range(5):
            if x >= interval[i] and x < interval[i + 1]:
                new_array.append(i * 0.25)
    return new_array

def calc_acc(arr1, arr2):
    match = 0
    all = len(arr1)
    for i in range(all):
        if arr1[i] == arr2[i]:
            match += 1
    return match / all

calc_acc(to_closest(output_scores), train_data['score'])

## 训练2

### 数据预处理：按照标签分类

In [ ]:
def divide_context(train_data, row_name="context"):
    all_contexts = train_data.loc[:, row_name].drop_duplicates()
    print(all_contexts)
    
    divided_data_list = []
    for context in all_contexts:
        new_data = train_data.loc[train_data.loc[:, row_name] == context]
        divided_data_list.append(new_data)
        
    return divided_data_list
    
raw_data_list = divide_context(train_data)

In [ ]:
def train_valid_split_by_context(divided_data_list):
    context_data_split_pairs = {}
    for context_data in divided_data_list:
        context_name = context_data.loc[0, 'context']

        context_processed_data = []
        for row in train_data.index:
            inp_example = InputExample(texts=[context_data.loc[row, 'anchor'],
                                             context_data.loc[row, 'target']],
                                      label=float(context_data.loc[row, 'score']))
            context_processed_data.append(inp_example)
            
        _, divided_context_data = split_list(context_processed_data, 
                                     group_num=5,
                                     shuffle=True,
                                     retain_left=False)

        train_context_processed_data = [x for l in divided_context_data[:-1] for x in l]
        valid_context_processed_data = divided_context_data[-1]
        context_data_split_pairs[context_pair] = [train_context_processed_data, valid_context_processed_data]\
        
split_data_list = train_valid_split_by_context(raw_data_list)

In [ ]:
def train_model_by_context(split_divided_data_list):
    model_list = {}
    for context in split_divided_data_list.keys():
        print("Loading model for %s..."%context)
        model = SentenceTransformer(model_name)

        train_dataloader = DataLoader(training_data,
                                      shuffle=True,
                                      batch_size=batch_size)
        train_loss = losses.CosineSimilarityLoss(model)
        evaluator = EmbeddingSimilarityEvaluator.from_input_examples(validation_data)
        warmup_steps = math.ceil(len(train_dataloader) * epoch_num * 0.1) # 10% of train data for warm-up

        print("Fit model...")
        model.fit(train_objectives=[(train_dataloader, train_loss)],
                  epochs=epoch_num,
                  evaluator=evaluator,
                  warmup_steps=warmup_steps)
        return model